In [1]:
import os, sys
import pandas as pd
import numpy as np
from os.path import join 

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [2]:
DataPath = r'D:\GitWork\titanic\data'
OutPath  = r'D:\GitWork\titanic\output'

TrainFile  = 'train.csv'
TestFile   = 'test.csv'
SubmitFile = 'Submission01.csv'

In [3]:
# Importing the data and displaying some rows
f_abspath = join(DataPath, TrainFile)
df_train = pd.read_csv(f_abspath)

display(df_train.info())
display(df_train.head(10))
display(df_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
features = ['PassengerId', 'Survived', 'Pclass', 'Sex', 'Fare']
df_data = df_train[features].copy()
display(df_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Fare         891 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 34.9+ KB


None

In [5]:
# Process Sex
uni_set = sorted(df_train.Sex.unique())
codes = [i for i in range(len(uni_set))]

dict_sex = dict(zip(uni_set, codes))
print('Sex dict:', dict_sex)

df_data['Sex_code'] = df_data['Sex'].map(dict_sex)
print(); display(df_data.info())

Sex dict: {'female': 0, 'male': 1}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Fare         891 non-null    float64
 5   Sex_code     891 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 41.9+ KB


None

Pclass dict: {1: 0, 2: 1, 3: 2}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  891 non-null    int64 
 1   Survived     891 non-null    int64 
 2   Pclass       891 non-null    int64 
 3   Sex          891 non-null    object
 4   Sex_code     891 non-null    int64 
 5   Pclass_code  891 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 41.9+ KB


None

Pclass dict: {1: 0, 2: 1, 3: 2}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  891 non-null    int64 
 1   Survived     891 non-null    int64 
 2   Pclass       891 non-null    int64 
 3   Sex          891 non-null    object
 4   Sex_code     891 non-null    int64 
 5   Pclass_code  891 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 41.9+ KB


None

Pclass dict: {1: 0, 2: 1, 3: 2}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  891 non-null    int64 
 1   Survived     891 non-null    int64 
 2   Pclass       891 non-null    int64 
 3   Sex          891 non-null    object
 4   Sex_code     891 non-null    int64 
 5   Pclass_code  891 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 41.9+ KB


None

Bins: [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 513]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Fare         891 non-null    float64 
 3   Bin          891 non-null    category
dtypes: category(1), float64(1), int64(2)
memory usage: 22.7 KB


None

count    13.000000
mean      0.579462
std       0.251182
min       0.199000
25%       0.425000
50%       0.586000
75%       0.758000
max       1.000000
Name: S1_rate, dtype: float64

'dict fare:'

{Interval(0, 10, closed='left'): 4,
 Interval(70, 80, closed='left'): 1,
 Interval(50, 60, closed='left'): 3}

AttributeError: 'DataFrame' object has no attribute 'Bin'

In [18]:
# Training 1
features = ['PassengerId', 'Survived', 'Sex','Pclass', 'Fare']
df_data = df_train[features].copy()

# For Sex
df_data['Sex_code'] = df_data['Sex'].map(dict_sex)

# For Fare
bins_fare = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 513] 
df_data['Bin'] = pd.cut(df_data['Fare'], bins_fare, right=False)
df_data['Fare_code'] = df_data.Bin.map(dict_fare)
display(df_data)

# inps = df_data.filter(features, axis=1)
# lbls = df_data['Survived']

# rf_model = RandomForestClassifier(
#     random_state=2, n_estimators=250, min_samples_split=20, oob_score=True
# )

# rf_model.fit(inps, lbls)
# print('oob score :%.5f' %(rf_model.oob_score_))

,PassengerId,Survived,Sex,Pclass,Fare,Sex_code,Bin,Fare_code
0,1,0,male,3,7.2500,1,"[0, 10)",NaN
1,2,1,female,1,71.2833,0,"[70, 80)",NaN
2,3,1,female,3,7.9250,0,"[0, 10)",NaN
3,4,1,female,1,53.1000,0,"[50, 60)",NaN
4,5,0,male,3,8.0500,1,"[0, 10)",NaN
...,...,...,...,...,...,...,...,...
886,887,0,male,2,13.0000,1,"[10, 20)",NaN
887,888,1,female,1,30.0000,0,"[30, 40)",NaN
888,889,0,female,3,23.4500,0,"[20, 30)",NaN
889,890,1,male,1,30.0000,1,"[30, 40)",NaN


In [8]:
# Prediction

# Convert Sex to Sex_code
f_abspath = join(DataPath, TestFile)
df_test = pd.read_csv(f_abspath)
display(df_test.info())
display(df_test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


None

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
# Process Fare column

# Fill null Fare 
df_test.Fare.fillna(df_test.Fare.mean())

0        7.8292
1        7.0000
2        9.6875
3        8.6625
4       12.2875
         ...   
413      8.0500
414    108.9000
415      7.2500
416      8.0500
417     22.3583
Name: Fare, Length: 418, dtype: float64

In [10]:
# Extract features
df_data = pd.DataFrame({
    'PassengerId': df_test.PassengerId,
    'Pclass': df_test.Pclass,
    'Sex_code': df_test['Sex'].map(dict_sex),
    'Fare'_code: 
})
display(df_data.info())


# Extract test columns
features = ['PassengerId', 'Sex_code','Pclass']
inps = df_data.filter(features, axis=1)

df_data['Survived'] = rf_model.predict(inps)
display(df_data.info())

f_abspath = join(OutPath, 'Submission01.csv')
out_features = ['PassengerId', 'Survived']
df_res = df_data[out_features]
df_res.to_csv(f_abspath, index=False)


SyntaxError: invalid syntax (<ipython-input-10-e416a2eb4d99>, line 6)